# Cálculo de NDVI 

En este notebook se generará la métrica de NDVI. 

NDVI (Normalized Difference Vegetation Index) is a widely used remote sensing index to measure vegetation health, density, and vigor. It works by leveraging the difference in reflectance properties of vegetation in the red and near-infrared (NIR) portions of the electromagnetic spectrum.

Healthy vegetation absorbs most of the red light (used in photosynthesis) and reflects a significant portion of near-infrared light. Unhealthy or sparse vegetation reflects more red light and less near-infrared light.

<div style="text-align: center;">
    <img src="Images/NVDI_formula.png" alt="Descripción de la imagen" width="500">
</div>
 




**IMPORTACION DE LIBRERIAS**

In [2]:
import os 
import ee 
import pandas as pd
import geemap
import requests
import plotly.express as px
import matplotlib.pyplot as plt
from rasterio.plot import show
import rasterio

# 1) Obtención de la dirección de trabajo
direction = os.getcwd()

# Remove the last part of the path
direction = os.path.dirname(direction)
direction

'/Users/orlandoandrade/Documents/Negocios/Satelites/selene'

## 1) Autentificación

In [3]:
# Dirección de la llave
service_account = direction + '/conf/local/gcp-for-data-science-397913-4fd843feede1.json'

# Autentificación 
credentials = ee.ServiceAccountCredentials(email=None, key_file=service_account)
ee.Initialize(credentials)

## 2) Lectura y procesamiento de imagenes  
Las imagenes satelitales se almacenaron en un bucket de GCP, por lo que serán leídas de ahi mismo.

API Reference:
 
  https://developers.google.com/earth-engine/apidocs/ee-image-loadgeotiff

### 2.1) Cálculo y exportación de imagenes con NDVI 
Las imagennes exportadas en 

In [4]:
# 1) Lectura del nombre de las imagenes en el bucket 
images_names = pd.read_csv(filepath_or_buffer=direction + "/data/02_intermediate/sentinel_images_names.csv")

# diccionario de imagenes 
images_dic = {}

# Importación del mapa base 
Map = geemap.Map()

# Path to the image in the GCS bucket
image_center = images_names["Images names"][0]
image_center   = f'gs://earth_engine_selene/sentinel2/Row_Data/{image_center}.tif'
    
# Load the image from the GCS bucket
image = ee.Image.loadGeoTIFF(uri=image_center)

# Central el mapa 
Map.centerObject(ee_object=image, zoom= 10)

# 2) Inicio del ciclo , se toma el pandas serie para iterar en él
for i in images_names["Images names"]:
    
    # 1) Descarga de imagenes 
    # Path to the image in the GCS bucket
    gcs_path = f'gs://earth_engine_selene/sentinel2/Row_Data/{i}.tif' # Crear más bien un GCP path base
    
    # Load the image from the GCS bucket
    image = ee.Image.loadGeoTIFF(gcs_path)
     
    # 2) Calculate NDVI (if bands are organized)
    image = image.normalizedDifference(bandNames=['B8', 'B4']).rename('NDVI')  # Renombre de la banda resultante (capa)
    print(f"NDVI Calculated for {i}")
    
    # 3) Generaación de URL 
    rgb_bands = ["NDVI"]

    # Determinación de los parámetros para la visualización
    ndvi_vis = {
    "min": 0,
    "max": 1,
   "palette": [
      "#FF0000",
      "#FF1700",
      "#FF2E00",
      "#FF4500",
      "#FF6500",
      "#FF8500",
      "#FFA500",
      "#FAC008",
      "#F6D80F",
      "#F1F117",
      "#D1ED19",
      "#B1E91B",
      "#91E51D",
      "#6FE120",
      "#53C918",
      "#37B110",
      "#1B9908",
      "#008000"],
    "bands": rgb_bands}

    url = image.getThumbURL(ndvi_vis)
    print(f"Thumbnail URL for {i}: {url}")
        
    # 4) Exportación de la imagen a PNG
    # Download the image and save it locally
    output_file = direction + f"/data/02_intermediate/NDVI/{i}.png"

    response = requests.get(url=url)
    if response.status_code == 200:
        with open(file=output_file, mode='wb') as f: #string "output_file" para exportar en esa direccion la imagen
            f.write(response.content) # Esto es lo que exportara que es el contenido de la respuesta, la imagen
        print(f"Image successfully downloaded as '{output_file}'")
    else:
        print(f"Error: Unable to download the image. HTTP status code {response.status_code}")
    
    images_dic[i]  = image # NO es necesario almacenarlo aqui (en un diccionario de python que terminara almacenando los datos en RAM localmente), en pequeño contexto no hay problema pero analizando mayores exteneiones puede tronar.  Hay que simplemente exportar los datos a un bucket y en el post procesamiento leer las imagenes una a una
    
    # 5) Añadimos la imagen al mapa html 
    Map.addLayer(ee_object=image,  vis_params= ndvi_vis , name=f'NDWI {i}', shown=True)

    # 6) Exportación de las imágen procesado geotiff al bucket de GCP, está imagen ya cotiene el cálculo de NDVI
    # Indicamos la tarea de exportacion
    task = ee.batch.Export.image.toCloudStorage(
    image=image,
    #description=f"Export", # Este es el nombre del archivo que se exportara , si vas a usar el filenameprexi no es necesario esto
    bucket='earth_engine_selene',  # Cambia esto al nombre de tu bucket
    fileNamePrefix=f"sentinel2/NDVI/NDVI_{i}", # Damos una dirección dentro del bucket y le damos nombre
    scale=10,  # Resolución en metros
    fileFormat='GeoTIFF')  # Formato de archivo

    # Inicio de la exportación
    task.start()  # Inicia la tarea de exportación
    print("Se exportó la imagen a Cloud Storage:", i)


# Añadimo una legenda, fuera del ciclo 
Map.add_colorbar(
ndvi_vis,
label="NDVI values",
layer_name="SRTM DEM",
orientation="vertical",
transparent_bg=True,
discrete=False)

# Exportación del HTML 
html_file = direction + "/data/07_model_output/NDVI/Map_NDVI.html"
Map.to_html(filename=html_file, title="My Map", width="100%", height="880px") # Exporacion a una html 

NDVI Calculated for Image_2024-10-04 00:00:00
Thumbnail URL for Image_2024-10-04 00:00:00: https://earthengine.googleapis.com/v1/projects/earthengine-legacy/thumbnails/53fd1d8058e885efc566fcde58012d53-af96fd206fd108d206b0913691f91394:getPixels
Image successfully downloaded as '/Users/orlandoandrade/Documents/Negocios/Satelites/selene/data/02_intermediate/NDVI/Image_2024-10-04 00:00:00.png'
Se exportó la imagen a Cloud Storage: Image_2024-10-04 00:00:00
NDVI Calculated for Image_2024-10-14 00:00:00
Thumbnail URL for Image_2024-10-14 00:00:00: https://earthengine.googleapis.com/v1/projects/earthengine-legacy/thumbnails/a5e910811644003e813e12260f40bf59-15257b191904c5bed9145d050248451f:getPixels
Image successfully downloaded as '/Users/orlandoandrade/Documents/Negocios/Satelites/selene/data/02_intermediate/NDVI/Image_2024-10-14 00:00:00.png'
Se exportó la imagen a Cloud Storage: Image_2024-10-14 00:00:00
NDVI Calculated for Image_2024-10-19 00:00:00
Thumbnail URL for Image_2024-10-19 00:00

**VISUALIZACION DEL MAPA**

In [5]:
#Map

### 3) Post procesamiento
En esta sección  lo que se busca es  tomas los geotiff generados del NDVI y generar un mapa a partir de estos. El mapa contendrá los siguientes elementos adicionales:

- Titulo
- Fecha de la imagen
- Leyenda con el valor de los colores
- Escala de la imagen

Para esto se utilizarán 2 librerías adicionales las cuales son: **Rasterio y matplolib**.

[Rasterio tiene un modulo de visualización (plot)](https://rasterio.readthedocs.io/en/stable/topics/plotting.html) que se convina con matplotlib para la visualización de archivos tipo raster

**IMPORTANTE:** Para que rasterio pueda hacer la lectura directamente desde el bucket es necesario que haga la autentificación con mi service account en variasbles del sistema.

In [6]:
# Autentificacion , variables del sistema
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] =  service_account

In [7]:
# Autentificacion con la libreria de google
from google.cloud import storage

client = storage.Client.from_service_account_json(json_credentials_path=service_account)
buckets = list(client.list_buckets())
buckets

# PROBAR  SI ESTO ES NECEARIO O NO !!!

[<Bucket: cv_portafolio_web_orlando>,
 <Bucket: earth_engine_selene>,
 <Bucket: test_data_science_orlando>]

In [8]:
import rasterio
from rasterio.plot import show
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, Normalize
from matplotlib_scalebar.scalebar import ScaleBar  # <-- for the scale bar

# Definir la paleta de colores personalizada
palette = [
    "#FF0000", "#FF1700", "#FF2E00", "#FF4500", "#FF6500", "#FF8500",
    "#FFA500", "#FAC008", "#F6D80F", "#F1F117", "#D1ED19", "#B1E91B",
    "#91E51D", "#6FE120", "#53C918", "#37B110", "#1B9908", "#008000"
]

cmap = ListedColormap(colors=palette) # Esto crea una lista de colores que puede ser integrado directamente en la figura

# Fijar el rango de valores de -1 a 1 para todas las imágenes
norm = Normalize(vmin=0, vmax=1) # IMPORTANTE: Hay que confirmar si es necesario que las imagenes las mostremos de -1 a 1. Por lo que entiendo de los valores de NDVI en las plantas su valores realmente van desde cercano de 0 a 1. Los valores negativos serán asociados a otras superfices. Por lo que no veo necesario inclurilas dentro de la escala (los valores negativos).

# Suponiendo que 'images_names' es un DataFrame o diccionario con la llave "Images names"
# Y que 'direction' es una variable con la ruta de salida
for i in images_names["Images names"]:
    print(i)

    # Ruta del archivo TIFF
    tiff_path = f"gs://earth_engine_selene/sentinel2/NDVI/NDVI_{i}.tif"

    # Lectura del archivo
    with rasterio.open(tiff_path) as src:
        # Lectura del archivo
        raster_data = src.read(1) # Creo que aqui se selecciona la banda de la imagen, hay que confirmar

        fig, ax = plt.subplots(figsize=(8, 6))

        # Plot del raster con la paleta personalizada y escala fija
        img = ax.imshow(raster_data, cmap=cmap, norm=norm)

        # Añadir una barra de colores (leyenda de NDVI)
        cbar = fig.colorbar(img, ax=ax)
        cbar.set_label('NDVI Value')

        # -------------------------------------------------
        # AÑADIR LA BARRA DE ESCALA (ScaleBar)
        # -------------------------------------------------
        # (1) Obtener la resolución de pixel (asumiendo un CRS en metros, por ejemplo UTM)
        pixel_size_x = abs(src.transform[0])  # tamaño de pixel en X (ej. metros)

        # (2) Crear y añadir la ScaleBar
        scalebar = ScaleBar(
            pixel_size_x,
            units='m',         # 'm' para metros, 'km' si desea kilómetros
            location='lower left',  # use espacio, no subrayado (p.ej. "lower left")
            length_fraction=0.25    # porcentaje del ancho del eje que ocupa la scale bar
        )
        ax.add_artist(scalebar)

        # Quitar los valores de los ejes X e Y
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_xlabel('')
        ax.set_ylabel('')

        # Añadir título
        ax.set_title(f"NDVI Map {i}")

        # Guardar la figura en un archivo .png
        fig.savefig(direction + f"/data/07_model_output/Sentinel/NDVI/static maps/NDVI_{i}.png")

        # Cerrar la figura para no mostrarla en pantalla
        plt.close(fig)

Image_2024-10-04 00:00:00
Image_2024-10-14 00:00:00
Image_2024-10-19 00:00:00
Image_2024-10-24 00:00:00
Image_2024-10-29 00:00:00
Image_2024-11-03 00:00:00
Image_2024-11-08 00:00:00
Image_2024-11-13 00:00:00
Image_2024-11-18 00:00:00
Image_2024-11-23 00:00:00
Image_2024-11-28 00:00:00
Image_2024-12-03 00:00:00
Image_2024-12-18 00:00:00
Image_2024-12-23 00:00:00
Image_2024-12-28 00:00:00
Image_2025-01-02 00:00:00


In [9]:
pixel_size_x

10.0

Referencia de como añadir elementos a los mapas:

https://www.youtube.com/watch?v=qiKns09X1Ao

In [10]:
## PENDIENTE : Establecer rangos de salud y hacer mas bien la visualizacion de manera categorica.
pendiente = "RANGOS DE SALUD" # Profundizar en lo que se muestra en la sigueinte imagen


### IMPORTANTE: Esta seccion va tener que tener sus categorias de valores por tipo de cultivo por semana!!!, ahorita lo voy a simplificar y lo dejare de manera generica de todo el año.

<div style="text-align: center;">
    <img src="Images/NDVI_categories.png" alt="Descripción de la imagen" width="500">
</div>


### 2.2)  NVDI promedio en el tiempo 
Para poder generar un scatter plot de los niveles de NVDI en la imagen a traves del tiempo hay que hacer un calculo de agregación (en este caso un promedio) de los valores de NVDI de los pixeles en cada imagen  y asociarlas

In [11]:
# Obtengo las llaves del diccionario 
keys  =  list(images_dic.keys()) # Recordemos que estas imagenes ya solo contienen la banda (indice de NVDI) calculado
    # Adicionalmente, recordemos que no se van a estar leyendo  las imagenes del diccionario, dado que ahí estaríua lamacenando los datos localmente. Cada vez que vaya a utilizarf las imagenes resultantres hay que leerlas irectamente dede GCP.

# Almacenamiento de la media del NVDI 
nvdi_values ={}

for i in keys:
    # Extracción de la imagen 
    ndvi_image = images_dic[i]
    
    # Calculo de la media de NVDI de toda la imagen 
    # Calculate mean using reduceRegion
    mean = ndvi_image.reduceRegion(
        reducer=ee.Reducer.mean(),
        scale=10,  # Pixel resolution (meters)
        maxPixels=1e9 # Esto es importante, es el número maximo de pixeles que se usarán para el cálculo 
    )
    # Esto es un ee diccionary 
    # The result is a earth engine diccionary, it is neccesary. In order to do this is neccesary to use the function get info()
    mean = mean.getInfo() # This returns a python dictionary
    
    # Almacenamiento de los resultados 
    nvdi_values[i] = mean["NDVI"] # Esta es la llave del diccionario

**RESULTADOS EN UN PANDAS DF**

In [12]:
# Convertimos los resultados a un pandas DF 
ndvi_mean_data = pd.DataFrame(list(nvdi_values.items()), columns =["Date", "NDVI"])
ndvi_mean_data

,Date,NDVI
0,Image_2024-10-04 00:00:00,0.136826
1,Image_2024-10-14 00:00:00,0.096483
2,Image_2024-10-19 00:00:00,0.118031
3,Image_2024-10-24 00:00:00,0.105557
4,Image_2024-10-29 00:00:00,0.109834
5,Image_2024-11-03 00:00:00,0.114662
6,Image_2024-11-08 00:00:00,0.111058
7,Image_2024-11-13 00:00:00,0.120712
8,Image_2024-11-18 00:00:00,0.121351
9,Image_2024-11-23 00:00:00,0.112846


**VISUALIZACION EN EL TIEMPO**

In [14]:
# Crear el gráfico con Plotly
fig = px.line(
    data_frame=ndvi_mean_data, x='Date', y='NDVI',
    markers=True,
    title='NDVI Over Time',
    labels={'Date': 'Date', 'NDVI': 'NDVI Value'}
)

# Mostrar el gráfico
#fig.show()

## 4) Exportación
Se exportará los resultados finales

In [16]:
# Exportación de tabla promedio del NVDI del área de interes
ndvi_mean_data.to_csv(path_or_buf=direction + "/data/07_model_output/Sentinel/NDVI/Medias_NDVI.csv", index=False)

# Exportación del grafico
fig.write_html(direction +"/data/07_model_output/Sentinel/NDVI/NDVI_time.html")